In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', delimiter=',', low_memory=False)
test = pd.read_csv('test.csv', delimiter=',', low_memory=False)

# 冒頭を表示して確認
train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [3]:
# trainデータを入力データとラベルに分割する
X = train.drop(['date_time'], axis=1).drop(['target_carbon_monoxide'], axis=1).drop(['target_benzene'], axis=1).drop(['target_nitrogen_oxides'], axis=1).values
Y_1 = train.target_carbon_monoxide.values  # carbon_monoxide(一酸化炭素)
Y_2 = train.target_benzene.values          # benzene(ベンゼン)
Y_3 = train.target_nitrogen_oxides.values  # nitrogen_oxides(窒素酸化物)

# 表示して確認
X, Y_1

(array([[1.3100e+01, 4.6000e+01, 7.5780e-01, ..., 1.0560e+03, 1.7428e+03,
         1.2934e+03],
        [1.3200e+01, 4.5300e+01, 7.2550e-01, ..., 1.1975e+03, 1.4499e+03,
         1.0109e+03],
        [1.2600e+01, 5.6200e+01, 7.5020e-01, ..., 1.0602e+03, 1.5861e+03,
         1.1170e+03],
        ...,
        [9.6000e+00, 3.4600e+01, 4.3100e-01, ..., 8.6190e+02, 8.8920e+02,
         1.1591e+03],
        [8.0000e+00, 4.0700e+01, 4.0850e-01, ..., 9.0850e+02, 9.1700e+02,
         1.2063e+03],
        [8.0000e+00, 4.1300e+01, 4.3750e-01, ..., 7.9710e+02, 8.8000e+02,
         1.2731e+03]]),
 array([2.5, 2.1, 2.2, ..., 1.6, 1.5, 1.4]))

In [4]:
# データを訓練データと検証データに分割する
def split_data(X, Y):
    # trainデータを訓練データと検証データに分割する
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    return x_train, x_test, y_train, y_test

In [5]:
# RMSLE カスタム評価関数 #####################
from keras import backend as K
msle = keras.metrics.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))

In [6]:
# DNNモデルをセット
def model_set(l1, l2, l1_drop, l2_drop):
    # DNNモデルを構築
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(x_train.shape[1],)),
        keras.layers.Dense(l1, activation='relu'),
        keras.layers.Dropout(l1_drop),
        keras.layers.Dense(l2, activation='relu'),
        keras.layers.Dropout(l2_drop),
        keras.layers.Dense(1, activation='linear')
    ])
    
    # モデルをコンパイルして構造を表示
    model.compile(optimizer='adam', #rmsprop
                  loss='mean_squared_error')
    model.summary()
    
    return model

# carbon_monoxide(一酸化炭素)について------------------------

In [7]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_1)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [8]:
# RNN(LSTM)モデルをセット
l1, l2, l1_drop, l2_drop = 65, 90, 0.028, 0.172
model_1 = model_set(l1, l2, l1_drop, l2_drop)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 65)                585       
_________________________________________________________________
dropout (Dropout)            (None, 65)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 90)                5940      
_________________________________________________________________
dropout_1 (Dropout)          (None, 90)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 91        
Total params: 6,616
Trainable params: 6,616
Non-trainable params: 0
______________________________________________________

In [9]:
# 学習させる
batch_size = 1607
epochs = 345
model_1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/345
4/4 [==============================] - 0s 5ms/step - loss: 25389.5988
Epoch 2/345
4/4 [==============================] - 0s 4ms/step - loss: 15258.7418
Epoch 3/345
4/4 [==============================] - 0s 4ms/step - loss: 11972.4309
Epoch 4/345
4/4 [==============================] - 0s 4ms/step - loss: 10429.4211
Epoch 5/345
4/4 [==============================] - 0s 4ms/step - loss: 8511.3268
Epoch 6/345
4/4 [==============================] - 0s 4ms/step - loss: 6715.8322
Epoch 7/345
4/4 [==============================] - 0s 4ms/step - loss: 5697.8114
Epoch 8/345
4/4 [==============================] - 0s 5ms/step - loss: 4889.4869
Epoch 9/345
4/4 [==============================] - 0s 4ms/step - loss: 4025.8550
Epoch 10/345
4/4 [==============================] - 0s 4ms/step - loss: 3304.9055
Epoch 11/345
4/4 [==============================] - 0s 5ms/step - loss: 2800.7151
Epoch 12/345
4/4 [==============================] - 0s 11ms/step - loss: 2447.3833
Epoch 13/345
4/4 [==

4/4 [==============================] - 0s 17ms/step - loss: 10.8076
Epoch 103/345
4/4 [==============================] - 0s 4ms/step - loss: 10.0235
Epoch 104/345
4/4 [==============================] - 0s 4ms/step - loss: 9.9556
Epoch 105/345
4/4 [==============================] - 0s 5ms/step - loss: 12.5108
Epoch 106/345
4/4 [==============================] - 0s 5ms/step - loss: 9.6008
Epoch 107/345
4/4 [==============================] - 0s 4ms/step - loss: 9.6683
Epoch 108/345
4/4 [==============================] - 0s 4ms/step - loss: 8.7524
Epoch 109/345
4/4 [==============================] - 0s 4ms/step - loss: 9.1220
Epoch 110/345
4/4 [==============================] - 0s 4ms/step - loss: 9.9606
Epoch 111/345
4/4 [==============================] - 0s 4ms/step - loss: 9.9829
Epoch 112/345
4/4 [==============================] - 0s 4ms/step - loss: 7.8235
Epoch 113/345
4/4 [==============================] - 0s 3ms/step - loss: 7.3149
Epoch 114/345
4/4 [==============================]

4/4 [==============================] - 0s 5ms/step - loss: 3.3978
Epoch 205/345
4/4 [==============================] - 0s 5ms/step - loss: 3.1213
Epoch 206/345
4/4 [==============================] - 0s 4ms/step - loss: 3.4420
Epoch 207/345
4/4 [==============================] - 0s 6ms/step - loss: 2.9141
Epoch 208/345
4/4 [==============================] - 0s 4ms/step - loss: 4.2066
Epoch 209/345
4/4 [==============================] - 0s 5ms/step - loss: 3.5391
Epoch 210/345
4/4 [==============================] - 0s 5ms/step - loss: 3.0482
Epoch 211/345
4/4 [==============================] - 0s 4ms/step - loss: 2.4431
Epoch 212/345
4/4 [==============================] - 0s 5ms/step - loss: 2.8954
Epoch 213/345
4/4 [==============================] - 0s 4ms/step - loss: 3.5111
Epoch 214/345
4/4 [==============================] - 0s 4ms/step - loss: 3.0477
Epoch 215/345
4/4 [==============================] - 0s 4ms/step - loss: 2.6887
Epoch 216/345
4/4 [==============================] - 0

4/4 [==============================] - 0s 5ms/step - loss: 2.0434
Epoch 307/345
4/4 [==============================] - 0s 5ms/step - loss: 1.9009
Epoch 308/345
4/4 [==============================] - 0s 4ms/step - loss: 1.8833
Epoch 309/345
4/4 [==============================] - 0s 5ms/step - loss: 1.6441
Epoch 310/345
4/4 [==============================] - 0s 4ms/step - loss: 3.5244
Epoch 311/345
4/4 [==============================] - 0s 5ms/step - loss: 1.6141
Epoch 312/345
4/4 [==============================] - 0s 3ms/step - loss: 2.0134
Epoch 313/345
4/4 [==============================] - 0s 4ms/step - loss: 1.6219
Epoch 314/345
4/4 [==============================] - 0s 4ms/step - loss: 1.8377
Epoch 315/345
4/4 [==============================] - 0s 5ms/step - loss: 1.8176
Epoch 316/345
4/4 [==============================] - 0s 4ms/step - loss: 1.7959
Epoch 317/345
4/4 [==============================] - 0s 4ms/step - loss: 1.3894
Epoch 318/345
4/4 [==============================] - 0

In [10]:
# 検証データへの精度
y_test_pred = model_1.predict(x_test)[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.180
r2: 0.818


# benzene(ベンゼン)について---------------------------------

In [11]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_2)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [12]:
# RNN(LSTM)モデルをセット
l1, l2, l1_drop, l2_drop = 91, 57, 0.035, 0.026
model_2 = model_set(l1, l2, l1_drop, l2_drop)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 91)                819       
_________________________________________________________________
dropout_2 (Dropout)          (None, 91)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 57)                5244      
_________________________________________________________________
dropout_3 (Dropout)          (None, 57)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 58        
Total params: 6,121
Trainable params: 6,121
Non-trainable params: 0
____________________________________________________

In [13]:
# 学習させる
batch_size = 76
epochs = 278
model_2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/278
75/75 [==============================] - 0s 912us/step - loss: 4185.0775
Epoch 2/278
75/75 [==============================] - 0s 1ms/step - loss: 604.3544
Epoch 3/278
75/75 [==============================] - 0s 1ms/step - loss: 220.6237
Epoch 4/278
75/75 [==============================] - 0s 1ms/step - loss: 143.3796
Epoch 5/278
75/75 [==============================] - 0s 886us/step - loss: 91.7114
Epoch 6/278
75/75 [==============================] - 0s 976us/step - loss: 59.4123
Epoch 7/278
75/75 [==============================] - 0s 1ms/step - loss: 46.6837
Epoch 8/278
75/75 [==============================] - 0s 1ms/step - loss: 27.7009
Epoch 9/278
75/75 [==============================] - 0s 976us/step - loss: 24.1609
Epoch 10/278
75/75 [==============================] - 0s 901us/step - loss: 22.7267
Epoch 11/278
75/75 [==============================] - 0s 1ms/step - loss: 16.0463
Epoch 12/278
75/75 [==============================] - 0s 1ms/step - loss: 17.3042
Epoch 13/2

75/75 [==============================] - 0s 982us/step - loss: 2.9112
Epoch 101/278
75/75 [==============================] - 0s 958us/step - loss: 2.8146
Epoch 102/278
75/75 [==============================] - 0s 952us/step - loss: 2.9184
Epoch 103/278
75/75 [==============================] - 0s 1ms/step - loss: 2.8727
Epoch 104/278
75/75 [==============================] - 0s 1ms/step - loss: 3.0640
Epoch 105/278
75/75 [==============================] - 0s 940us/step - loss: 2.8680
Epoch 106/278
75/75 [==============================] - 0s 991us/step - loss: 2.9088
Epoch 107/278
75/75 [==============================] - 0s 871us/step - loss: 3.2265
Epoch 108/278
75/75 [==============================] - 0s 863us/step - loss: 2.8795
Epoch 109/278
75/75 [==============================] - 0s 893us/step - loss: 2.8694
Epoch 110/278
75/75 [==============================] - 0s 865us/step - loss: 2.9245
Epoch 111/278
75/75 [==============================] - 0s 1ms/step - loss: 2.7929
Epoch 112/27

75/75 [==============================] - 0s 921us/step - loss: 2.5998
Epoch 199/278
75/75 [==============================] - 0s 1ms/step - loss: 2.2085
Epoch 200/278
75/75 [==============================] - 0s 1ms/step - loss: 2.3225
Epoch 201/278
75/75 [==============================] - 0s 1ms/step - loss: 2.2547
Epoch 202/278
75/75 [==============================] - 0s 957us/step - loss: 2.4580
Epoch 203/278
75/75 [==============================] - 0s 1ms/step - loss: 2.1991
Epoch 204/278
75/75 [==============================] - 0s 1ms/step - loss: 2.1168
Epoch 205/278
75/75 [==============================] - 0s 1ms/step - loss: 2.4194
Epoch 206/278
75/75 [==============================] - 0s 1ms/step - loss: 2.7681
Epoch 207/278
75/75 [==============================] - 0s 1ms/step - loss: 2.3364
Epoch 208/278
75/75 [==============================] - 0s 1ms/step - loss: 2.4608
Epoch 209/278
75/75 [==============================] - 0s 986us/step - loss: 2.4238
Epoch 210/278
75/75 [===

In [14]:
# 検証データへの精度
y_test_pred = model_2.predict(x_test)[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.156
r2: 0.961


# nitrogen_oxides(窒素酸化物)について-------------------------

In [15]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_3)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [16]:
# RNN(LSTM)モデルをセット
l1, l2, l1_drop, l2_drop = 10, 100, 0.0, 0.209
model_3 = model_set(l1, l2, l1_drop, l2_drop)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                90        
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               1100      
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 1,291
Trainable params: 1,291
Non-trainable params: 0
____________________________________________________

In [17]:
# 学習させる
batch_size = 1185
epochs = 455
model_3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/455
5/5 [==============================] - 0s 4ms/step - loss: 195395.6719
Epoch 2/455
5/5 [==============================] - 0s 6ms/step - loss: 97554.6784
Epoch 3/455
5/5 [==============================] - 0s 3ms/step - loss: 59368.4570
Epoch 4/455
5/5 [==============================] - 0s 3ms/step - loss: 55129.3509
Epoch 5/455
5/5 [==============================] - 0s 3ms/step - loss: 56099.1725
Epoch 6/455
5/5 [==============================] - 0s 4ms/step - loss: 51664.1341
Epoch 7/455
5/5 [==============================] - 0s 3ms/step - loss: 44176.5365
Epoch 8/455
5/5 [==============================] - 0s 5ms/step - loss: 43374.9701
Epoch 9/455
5/5 [==============================] - 0s 3ms/step - loss: 40091.2734
Epoch 10/455
5/5 [==============================] - 0s 3ms/step - loss: 36933.1484
Epoch 11/455
5/5 [==============================] - 0s 3ms/step - loss: 33939.5846
Epoch 12/455
5/5 [==============================] - 0s 4ms/step - loss: 32067.6904
Epoch 13/455

5/5 [==============================] - 0s 3ms/step - loss: 12061.7144
Epoch 100/455
5/5 [==============================] - 0s 2ms/step - loss: 11826.3270
Epoch 101/455
5/5 [==============================] - 0s 3ms/step - loss: 12308.0584
Epoch 102/455
5/5 [==============================] - 0s 3ms/step - loss: 11710.0355
Epoch 103/455
5/5 [==============================] - 0s 2ms/step - loss: 12138.1051
Epoch 104/455
5/5 [==============================] - 0s 3ms/step - loss: 12204.5576
Epoch 105/455
5/5 [==============================] - 0s 3ms/step - loss: 11427.0835
Epoch 106/455
5/5 [==============================] - 0s 2ms/step - loss: 12001.4421
Epoch 107/455
5/5 [==============================] - 0s 2ms/step - loss: 11532.9160
Epoch 108/455
5/5 [==============================] - 0s 3ms/step - loss: 11859.2152
Epoch 109/455
5/5 [==============================] - 0s 6ms/step - loss: 12437.3626
Epoch 110/455
5/5 [==============================] - 0s 5ms/step - loss: 11536.9705
Epoch 

5/5 [==============================] - 0s 3ms/step - loss: 10848.8743
Epoch 197/455
5/5 [==============================] - 0s 3ms/step - loss: 10116.4824
Epoch 198/455
5/5 [==============================] - 0s 3ms/step - loss: 10957.5807
Epoch 199/455
5/5 [==============================] - 0s 3ms/step - loss: 10174.9095
Epoch 200/455
5/5 [==============================] - 0s 3ms/step - loss: 10498.5628
Epoch 201/455
5/5 [==============================] - 0s 3ms/step - loss: 9911.0150
Epoch 202/455
5/5 [==============================] - 0s 3ms/step - loss: 9958.1369
Epoch 203/455
5/5 [==============================] - 0s 3ms/step - loss: 9813.0270
Epoch 204/455
5/5 [==============================] - 0s 3ms/step - loss: 10262.5876
Epoch 205/455
5/5 [==============================] - 0s 2ms/step - loss: 9891.9494
Epoch 206/455
5/5 [==============================] - 0s 3ms/step - loss: 10443.1100
Epoch 207/455
5/5 [==============================] - 0s 3ms/step - loss: 9447.7402
Epoch 208/4

5/5 [==============================] - 0s 3ms/step - loss: 9187.2251
Epoch 295/455
5/5 [==============================] - 0s 3ms/step - loss: 9968.6847
Epoch 296/455
5/5 [==============================] - 0s 3ms/step - loss: 10107.5579
Epoch 297/455
5/5 [==============================] - 0s 3ms/step - loss: 9748.8436
Epoch 298/455
5/5 [==============================] - 0s 2ms/step - loss: 9865.6748
Epoch 299/455
5/5 [==============================] - 0s 3ms/step - loss: 9785.3208
Epoch 300/455
5/5 [==============================] - 0s 3ms/step - loss: 9250.2338
Epoch 301/455
5/5 [==============================] - 0s 3ms/step - loss: 9429.8283
Epoch 302/455
5/5 [==============================] - 0s 3ms/step - loss: 10353.6108
Epoch 303/455
5/5 [==============================] - 0s 3ms/step - loss: 9642.9430
Epoch 304/455
5/5 [==============================] - 0s 3ms/step - loss: 9933.0160
Epoch 305/455
5/5 [==============================] - 0s 3ms/step - loss: 9727.9810
Epoch 306/455
5/

5/5 [==============================] - 0s 3ms/step - loss: 9569.9647
Epoch 393/455
5/5 [==============================] - 0s 3ms/step - loss: 9570.0345
Epoch 394/455
5/5 [==============================] - 0s 3ms/step - loss: 9959.1991
Epoch 395/455
5/5 [==============================] - 0s 3ms/step - loss: 9988.6646
Epoch 396/455
5/5 [==============================] - 0s 2ms/step - loss: 9151.4466
Epoch 397/455
5/5 [==============================] - 0s 3ms/step - loss: 9612.6006
Epoch 398/455
5/5 [==============================] - 0s 3ms/step - loss: 9584.8753
Epoch 399/455
5/5 [==============================] - 0s 3ms/step - loss: 9964.5044
Epoch 400/455
5/5 [==============================] - 0s 2ms/step - loss: 8730.2738
Epoch 401/455
5/5 [==============================] - 0s 3ms/step - loss: 10202.9220
Epoch 402/455
5/5 [==============================] - 0s 3ms/step - loss: 9552.8089
Epoch 403/455
5/5 [==============================] - 0s 3ms/step - loss: 9962.1548
Epoch 404/455
5/5

In [18]:
# 検証データへの精度
y_test_pred = model_3.predict(x_test)[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.255
r2: 0.790


# 提出データへの適用

In [19]:
# testデータを適用できる形に変換する
X_sb = test.drop(['date_time'], axis=1).values

# 表示して確認
X_sb

array([[8.0000e+00, 4.1300e+01, 4.3750e-01, ..., 7.9710e+02, 8.8000e+02,
        1.2731e+03],
       [5.1000e+00, 5.1700e+01, 4.5640e-01, ..., 6.8790e+02, 9.7280e+02,
        1.7140e+03],
       [5.8000e+00, 5.1500e+01, 4.6890e-01, ..., 6.9370e+02, 9.4190e+02,
        1.3008e+03],
       ...,
       [2.6600e+01, 1.9000e+01, 6.4060e-01, ..., 5.7280e+02, 1.2534e+03,
        1.0811e+03],
       [2.9100e+01, 1.2700e+01, 5.1390e-01, ..., 7.0200e+02, 1.0098e+03,
        8.0850e+02],
       [2.7900e+01, 1.3500e+01, 5.0280e-01, ..., 6.0820e+02, 1.0613e+03,
        8.1600e+02]])

In [20]:
# 各目的変数を回帰
predict_1 = pd.DataFrame(model_1.predict(X_sb.reshape(-1, 8, 1))[:,0])
predict_2 = pd.DataFrame(model_2.predict(X_sb.reshape(-1, 8, 1))[:,0])
predict_3 = pd.DataFrame(model_3.predict(X_sb.reshape(-1, 8, 1))[:,0])

# データの形を確認
predict_1.shape, predict_2.shape, predict_3.shape

((2247, 1), (2247, 1), (2247, 1))

In [21]:
# 結果を提出形式に変形
df_predictions = pd.concat([predict_1, predict_2, predict_3],axis=1)
df_predictions.columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
submit_data = pd.concat([test.date_time, df_predictions],axis=1)
submit_data = submit_data.set_index('date_time')
submit_data.head()

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
date_time,,,
2011-01-01 00:00:00,2.152377,5.027287,206.295807
2011-01-01 01:00:00,3.322798,7.214564,340.393219
2011-01-01 02:00:00,2.558191,6.462319,395.852386
2011-01-01 03:00:00,2.512103,7.178486,405.069794
2011-01-01 04:00:00,1.820262,5.144939,285.423737


In [22]:
# CSVファイルとして出力
submit_data.to_csv("submission_DNN_4.csv")